### Filtro basado en contenido:

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Cargar el conjunto de datos
df = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', sep='::', engine='python', header= None)
cols = ['movie_id', 'title', 'genres']
df.columns = cols
df['genres'] = df['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])
df['genres'] = df['genres'].apply(lambda x: ' '.join(x))

In [ ]:
# Crear la matriz de características
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['genres'])

svd = TruncatedSVD(n_components=31)  # Especifica el número de componentes principales deseados
tfidf_matrix_svd = svd.fit_transform(tfidf_matrix)

cosine_sim = cosine_similarity(tfidf_matrix_svd, tfidf_matrix_svd)

In [ ]:
# Función para obtener recomendaciones
def get_recommendations(movie_title, cosine_sim, df):
    if movie_title not in df['title'].values:
        print(f'Error: La película "{movie_title}" no se encuentra en el DataFrame.')
        return None

    movie_index = df[df['title'] == movie_title].index[0]
    similarity_scores = list(enumerate(cosine_sim[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_similar_movies = [df['title'][i[0]] for i in similarity_scores[1:6]]  # Obtener las 5 películas más similares
    return top_similar_movies

In [ ]:
# Ejemplo de uso
movie_title = "The Dark Knight (2008)"
top_similar_movies = get_recommendations(movie_title, cosine_sim, df)
print(f'Películas similares a "{movie_title}":')
for movie in top_similar_movies:
    print(movie)

### Filtro colaborativo basado en usuario:

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Cargar el conjunto de datos de películas
url = 'https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat'
movies_df = pd.read_csv(url, sep='::', header=None, engine='python', encoding='latin1')
movies_df.columns = ['movie_id', 'title', 'genre']

url = 'https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat'
ratings_df = pd.read_csv(url, sep='::', header=None, engine='python', encoding='latin1')
ratings_df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [4]:
# Reemplazar los valores negativos en las columnas 'user_id' y 'movie_id'
ratings_df['user_id'] = ratings_df['user_id'].abs()
ratings_df['movie_id'] = ratings_df['movie_id'].abs()
movies_df['movie_id'] = movies_df['movie_id'].abs()

In [5]:
# Crear una matriz de utilidad de usuarios y películas
utility_matrix = ratings_df.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

C:\Users\tcardenas\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\reshape.py:130: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [ ]:
# Calcular la similitud del coseno entre los usuarios
user_similarity = cosine_similarity(utility_matrix)



In [ ]:
# Función para obtener las películas recomendadas para un usuario específico
def get_user_recommendations(user_id, num_recommendations=10):
    # Obtener la fila de similitud del usuario
    user_similarities = user_similarity[user_id]

    # Calcular el promedio ponderado de las clasificaciones de películas de los usuarios similares
    weighted_ratings = np.dot(user_similarities, utility_matrix.values) / np.sum(np.abs(user_similarities))

    # Excluir las películas que el usuario ya ha visto
    watched_movies = utility_matrix.loc[user_id] > 0
    weighted_ratings[watched_movies] = 0

    # Ordenar las películas recomendadas por la clasificación ponderada
    movie_ids = np.argsort(weighted_ratings)[::-1][:num_recommendations]

    # Obtener los títulos de las películas recomendadas
    recommended_movies = movies_df[movies_df['movie_id'].isin(movie_ids)]['title']

    return recommended_movies



In [ ]:
# Obtener las recomendaciones para un usuario específico (por ejemplo, usuario ID 1)
user_id = 1
recommended_movies = get_user_recommendations(user_id)
print(f"Películas recomendadas para el usuario {user_id}:")
print(recommended_movies)